In [1]:
import pandas as pd
import copy
import ast

pd.set_option('display.max_columns', None) # display all columns

In [2]:
df = pd.read_csv('final_reviews.csv')
# df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'index', 'Unnamed: 0.1.1', 'Unnamed: 0_x', 'Unnamed: 0_y'], inplace=True)
df_standby = df.copy()  # just in case i screw up the df columns

/Users/tanyh/.pyenv/versions/3.9.2/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head(3)

,Unnamed: 0,category,description,title,also_buy,brand,feature,rank,also_view,main_cat,similar_item,price,asin,rankElectronics,Rating,vote,verified,reviewText,summary,reviewClean,summaryClean,reviewCleanLemm,tb_score,afinn_score,vader_com,label,label_cat,summaryReview
0,0,"['Electronics', 'Television & Video', 'Streami...",['The future of streaming is here. Brilliant p...,Roku 4 | HD and 4K UHD Streaming Media Player ...,[],Roku,"['2,500+ streaming channls. Choose from the bi...","['>#5,130 in Electronics (See top 100)', '>#72...","['B0779LM75Y', 'B075XN1NZC', 'B075XMZMWY', 'B0...",All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...",7.45,B015YF5YIS,5130.0,3.0,NaN,True,Works until it quits. Had to send mine in twic...,Three Stars,works quits send mine twice repair quick turn ...,three stars,work quits send mine twice repair quick turn a...,0.333333,0.0,0.0000,negative,0,three stars work quits send mine twice repair ...
1,1,"['Electronics', 'Accessories &amp; Supplies', ...",['Prevent your relay rack from becoming an unm...,C2G/Cables to Go 03748 Vertical Cable Manageme...,"['B000FAH6I4', 'B00O63LWW2', 'B00006MJFL', 'B0...",C2G/Cables To Go,['Organize your rackmount to look professional...,"['>#98,575 in Electronics (See Top 100 in Elec...",[],All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...",2.23,B00006MJFO,98575.0,5.0,NaN,False,way cheaper then others in the same area i lov...,wokrs great used it several times,way cheaper others area love cut half use make...,wokrs great used several times,way cheaper others area love cut half use make...,0.166667,2.0,0.4767,positive,1,wokrs great used several times way cheaper oth...
2,2,"['Electronics', 'Home Audio', 'Speakers', 'Boo...",[],Edifier S1000DB Audiophile Active Bookshelf Sp...,"['B0725GZQFZ', 'B07CT4H1DT', 'B00GA9COMM', 'B0...",Edifier,['BLUETOOTH 4.0 aptX connectivity Stream high...,"['>#7,075 in Electronics (See Top 100 in Elect...",[],Home Audio & Theater,"class=""a-bordered a-horizontal-stripes a-spa...",10.84,B01CDIS6M0,7075.0,5.0,4.0,True,After searching around and reading reviews on ...,Edifier S1000DB Speakers are Great!,searching around reading reviews various speak...,edifier db speakers great,search around reading review various speaker l...,0.252828,7.0,0.9601,positive,1,edifier db speakers great search around readin...


In [4]:
df.dtypes

Unnamed: 0           int64
category            object
description         object
title               object
also_buy            object
brand               object
feature             object
rank                object
also_view           object
main_cat            object
similar_item        object
price              float64
asin                object
rankElectronics    float64
Rating             float64
vote                object
verified              bool
reviewText          object
summary             object
reviewClean         object
summaryClean        object
reviewCleanLemm     object
tb_score           float64
afinn_score        float64
vader_com          float64
label               object
label_cat            int64
summaryReview       object
dtype: object

In [5]:
### Data Cleaning: because when NaN, it means 0 votes (because there is no 0 vote in df)

df['vote'] = df['vote'].fillna(0)

## 1. Feature Engineering

#### Step 1: Aggregate len and count for column variables


In [4]:
# changing vote into float
df['vote'] = df['vote'].apply(lambda s: str(s))
df['vote'] = df['vote'].apply(lambda s: s.replace(',',''))  # for thousands, it is 1,702, hence need remove ,
df['vote'] = df['vote'].astype(float)

# aggregate len and count
df['category_count'] = df.category.apply(lambda x: len(ast.literal_eval(x)))
df['title_len'] = df.title.apply(lambda x: len(x.split()))
df['also_buy_count'] = df.also_buy.apply(lambda x: len(ast.literal_eval(x)))
df['also_view_count'] = df.also_view.apply(lambda x: len(ast.literal_eval(x)))
df['summary_len'] = df.summary.apply(lambda x: len(x.split()))
df['review_text_len'] = df.reviewText.apply(lambda x: len(x.split()))

##### Problems Faced

1. features, description all has html elements in it, how to remove them (challenge)


In [7]:
#df['description_len'] = df.description.apply(lambda x: len(ast.literal_eval(x)[0]))
#df['feature_len'] = df.feature.apply(lambda x: len(ast.literal_eval(x)[0]))

#### Step 2: Aggregating Dataset based on asin

Elements to aggregate<br>
1. Rating
2. Verified (based on the reviews)
3. All the sentiment scores

In [8]:
#### Creating new dataframe for Stage 2 (predict sales rank)

# aggregate rating, review_count, also_buy, also_view
new_df = df.groupby('asin')[['Rating']].mean().reset_index().rename(columns={"Rating":"rating"}) # Rating
vote_count_df = df.groupby('asin')[['vote']].mean().reset_index() # vote
review_count_df = df.groupby('asin')[['Rating']].count().reset_index().rename(columns={"Rating":"review_count"}) # Review count
review_text_len_df = df.groupby('asin')[['review_text_len']].mean().reset_index() # review Text len
also_buy_count_df = df.groupby('asin')[['also_buy_count']].count().reset_index()
also_view_count_df = df.groupby('asin')[['also_view_count']].count().reset_index()
category_count_df = df.groupby('asin')[['category_count']].count().reset_index()

# aggregate number of true and false in verified
verified_true_df = df.groupby('asin')[['verified']].sum().reset_index().rename(columns={"verified":"true_count"}) # total true: True = 1, False = 0
verified_total_df = df.groupby('asin')[['verified']].count().reset_index().rename(columns={"verified":"total_count"}) # total verified
verified_true_df.sort_values(by=['asin'], inplace=True)
verified_total_df.sort_values(by=['asin'], inplace=True)
verified_true_df = pd.merge(verified_true_df, verified_total_df, on="asin", how='left')
verified_true_df['verified_true_ratio'] = verified_true_df['true_count']/verified_true_df['total_count']
verified_true_df.drop(columns=['true_count', 'total_count'], inplace=True)

# aggregate brand, main_cat, price, rankElectronics
title_df = df.groupby('asin')['title'].apply(lambda x: list(x)[0]).to_frame().reset_index()
title_len_df = df.groupby('asin')['title_len'].apply(lambda x: list(x)[0]).to_frame().reset_index()
summary_len_df = df.groupby('asin')['summary_len'].apply(lambda x: list(x)[0]).to_frame().reset_index()
brand_df = df.groupby('asin')['brand'].apply(lambda x: list(x)[0]).to_frame().reset_index()
main_cat_df = df.groupby('asin')['main_cat'].apply(lambda x: list(x)[0]).to_frame().reset_index()
price_df = df.groupby('asin')['price'].apply(lambda x: list(x)[0]).to_frame().reset_index()
rank_df = df.groupby('asin')['rankElectronics'].apply(lambda x: list(x)[0]).to_frame().reset_index()
percentage_positive_df = df.groupby(['asin']).apply(lambda x: x['label_cat'].sum()/len(x)).to_frame().reset_index().rename(columns={0: 'percentage_positive'})
tb_score_df = df.groupby('asin')[['tb_score']].mean().reset_index()
afinn_score_df = df.groupby('asin')[['afinn_score']].mean().reset_index()
vader_score_df = df.groupby('asin')[['vader_com']].mean().reset_index()

# merge into new df
new_df = pd.merge(new_df, title_df, on="asin", how='left')
new_df = pd.merge(new_df, title_len_df, on="asin", how='left')
new_df = pd.merge(new_df, vote_count_df, on="asin", how='left')
new_df = pd.merge(new_df, summary_len_df, on="asin", how='left')
new_df = pd.merge(new_df, category_count_df, on="asin", how='left')
new_df = pd.merge(new_df, review_count_df, on="asin", how='left')
new_df = pd.merge(new_df, verified_true_df, on="asin", how='left')
new_df = pd.merge(new_df, also_buy_count_df, on="asin", how='left')
new_df = pd.merge(new_df, also_view_count_df, on="asin", how='left')
new_df = pd.merge(new_df, brand_df, on="asin", how='left')
new_df = pd.merge(new_df, main_cat_df, on="asin", how='left')
new_df = pd.merge(new_df, price_df, on="asin", how='left')
new_df = pd.merge(new_df, rank_df, on="asin", how='left')
new_df = pd.merge(new_df, review_text_len_df, on="asin", how='left')
new_df = pd.merge(new_df, percentage_positive_df, on="asin", how='left')
new_df = pd.merge(new_df, tb_score_df, on="asin", how='left')
new_df = pd.merge(new_df, afinn_score_df, on="asin", how='left')
new_df = pd.merge(new_df, vader_score_df, on="asin", how='left')

In [9]:
new_df

,asin,rating,title,title_len,vote,summary_len,category_count,review_count,verified_true_ratio,also_buy_count,also_view_count,brand,main_cat,price,rankElectronics,review_text_len,percentage_positive,tb_score,afinn_score,vader_com
0,B00000J1EI,4.428571,"Maxell GX-MP 120 Camcorder tapes, 3 Pack",7,NaN,2,7,7,1.000000,7,7,Maxell,Home Audio & Theater,35.89,15985.0,7.571429,0.857143,0.260823,2.857143,0.397443
1,B00000JD3C,4.333333,Sony 3MC-60B Microcassette - 3 Pack,6,7.000000,2,18,18,1.000000,18,18,Sony,Home Audio & Theater,8.95,8344.0,21.111111,0.722222,0.243182,2.555556,0.326794
2,B00000JD4T,4.000000,ETON G2000A AM/FM Shortwave Radio (Discontinue...,8,3.000000,5,2,2,0.000000,2,2,Eton,Home Audio & Theater,7.69,186267.0,149.500000,1.000000,0.114865,6.500000,0.090350
3,B00000JMRV,4.200000,Grundig M100 Portable Radio (Discontinued by M...,7,NaN,5,5,5,0.400000,5,5,Eton,Home Audio & Theater,7.67,142184.0,40.600000,0.600000,0.345846,2.800000,0.320060
4,B00000JMRX,2.500000,ETON YB300PE AM/FM Shortwave Radio (Discontinu...,8,NaN,2,2,2,0.000000,2,2,Eton,Home Audio & Theater,7.69,509371.0,259.500000,0.000000,0.041331,-7.000000,-0.277250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7092,B01HGME54Q,4.600000,Mini Skater Circle Ethernet Nylon Wire Cable C...,15,NaN,2,5,5,1.000000,5,5,Mini Skater,Home Audio & Theater,6.50,42825.0,15.200000,0.800000,0.510387,2.800000,0.509680
7093,B01HGYMEKQ,4.375000,Yamaha BD-S681 Blu-ray Disc Player,5,6.142857,6,8,8,1.000000,8,8,Yamaha Audio,All Electronics,13.18,21487.0,83.625000,0.875000,0.167470,8.875000,0.360875
7094,B01HI829TG,3.666667,Camecho Wireless Car Vehicle Backup Camera Sys...,26,3.000000,12,9,9,1.000000,9,9,Camecho,Car Electronics,4.96,50724.0,11.333333,0.888889,0.340926,1.555556,0.222922
7095,B01HIL1XZY,3.812500,Studebaker 4330975871 Black Retro SB2000 Repli...,12,15.222222,2,16,16,1.000000,16,16,Studebaker,All Electronics,1.86,17509.0,83.000000,0.750000,0.285313,7.437500,0.589544


In [10]:
new_df.to_csv("aggregated_reviews_meta.csv")